In [1]:
import logging
#Logging is a means of tracking events that happen when some software runs
# Create and configure logger
logging.basicConfig(filename="../logs/newfile.log",
                    format='%(asctime)s %(message)s',
                    filemode='w')
 
# Creating an object
logger = logging.getLogger()
 
# Setting the threshold of logger to DEBUG
logger.setLevel(logging.DEBUG)
 
# Test messages
logger.debug("Harmless debug Message")
logger.info("Just an information")
logger.warning("Its a Warning")
logger.error("Did you try to divide by zero")
logger.critical("Internet is down")

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

sns.set_style("darkgrid")
%matplotlib inline
import sys
sys.path.insert(0,'../scripts/')

C:\Users\kachase\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
from data_loader import load_df_from_csv
from data_struct import DataInfo
from cleaner import DataCleaner
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.preprocessing import LabelEncoder

C:\Users\kachase\Anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [4]:
df = pd.read_csv("../data/train_processed.csv")

test = pd.read_csv("../data/test_processed.csv")

In [5]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

In [6]:
#cat_data = df.select_dtypes(exclude=[np.number])
#print(cat_data)

In [7]:
df = pd.get_dummies(df, columns=['StoreType', 'Assortment', 'PromoInterval'])
test = pd.get_dummies(test, columns=['StoreType','Assortment','PromoInterval'])

In [8]:
df.head()

,Unnamed: 0,Store,DayOfWeek,Sales,Open,Promo,SchoolHoliday,Day,Year,DayOfYear,...,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,PromoInterval_0,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec"
0,0,1,3,5530,1,0,1,2,2013,2,...,0,1,0,1,0,0,1,0,0,0
1,1,1,4,4327,1,0,1,3,2013,3,...,0,1,0,1,0,0,1,0,0,0
2,2,1,5,4486,1,0,1,4,2013,4,...,0,1,0,1,0,0,1,0,0,0
3,3,1,6,4997,1,0,1,5,2013,5,...,0,1,0,1,0,0,1,0,0,0
4,4,1,1,7176,1,1,1,7,2013,7,...,0,1,0,1,0,0,1,0,0,0


In [9]:
from pandas import Series, DataFrame
train_dict = dict(list(df.groupby("Store")))
test_dict = dict(list(test.groupby("Store")))
predictions = Series()


In [10]:
pip install mlflow

  Using cached https://files.pythonhosted.org/packages/f0/ca/2c0186954b2f162b07efe8d82dbb858bbb69ca2373af9fc02c736c52cdbe/mlflow-1.26.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/05/40/d836d55fb3f467243ee839ab7b814822fda522cd395fa41e282684e71ee5/sqlparse-0.4.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b3/e2/8d48220731b7279911c43e95cd182961a703b939de6822b00de3ea0d3159/alembic-1.7.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/88/6b/572b2590fd55114118bf08bde63c0a421dcc82d593700f3e2ad89908a8a9/querystring_parser-1.2.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/dc/7e/157c03d180a1180cb4df6b765abad7076a13d6072844091c706926bbf659/databricks-cli-0.16.6.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\kachase\Anaconda3\lib\site-packages\setuptools\__init__.py"

Command "python setup.py egg_info" failed with error code 1 in C:\Users\kachase\AppData\Local\Temp\pip-install-rwfpjeea\databricks-cli\


In [12]:
for i in test_dict:
    df = train_dict[i]
    X_train = df.drop(["Sales", "Store"], axis=1)
    Y_train = df["Sales"]
    Y_train = np.log(Y_train + 1)
    X_test = test_dict[i].copy()
    store_ids = X_test["Id"]
    X_test.drop(["Id", "Store"], axis=1, inplace=True)

    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
    import mlflow
    mlflow.autolog()

    rf = RandomForestRegressor(n_estimators=30,max_depth=8)
    #rf.fit(X_train, Y_train)
    #Y_pred = randforestreg.predict(X_test)

    #Y_pred = np.exp(Y_pred) - 1

    #predictions = predictions.append(Series(Y_pred, index=store_ids))autolog_run = mlflow.last_active_run()
    autolog_run = mlflow.last_active_run()



ModuleNotFoundError: No module named 'mlflow'

In [ ]:
predictions = predictions.append(Series(0, index=closed_store_ids))

submission = pd.DataFrame({"Id": predictions.index, "Sales": predictions.values})
submission.to_csv("predictions/randomforestregressor-independent-log.csv", index=False)